# Pre Proccessing

In [3]:
import os
import re
import pandas as pd

In [4]:
import os
import argparse
import subprocess
from Bio import SeqIO

In [5]:
from itertools import repeat
from multiprocessing import Pool, freeze_support

In [6]:
InDir = "/mnt/d/Lab/16S-Taxa-Phlyo/test_data"

In [26]:
file = "DA_164__1492R__F06_2009041343G.ab1"

In [27]:
filestr = re.sub("[-()[\]]", "_", file)

In [28]:
filestr

'DA_164__1492R__F06_2009041343G.ab1'

In [29]:
filestr = re.sub("\.", "__", filestr)

In [30]:
filestr

'DA_164__1492R__F06_2009041343G__ab1'

In [7]:
def manifest(InDir):
    R1List = []
    R2List = []
    IdList = []
    df = pd.DataFrame()
    # Find all forward sequence files and reverse sequence files
    for subdir, dirs, files in os.walk(InDir):
        for file in files:
            filePath = os.path.join(subdir, file)
            if file.endswith(".ab1") and os.path.getsize(filePath) > 0:
                SampleInfo = file.split("__")
                if SampleInfo[1].endswith("F"):
                    R1List.append(filePath)
                    IdList.append(SampleInfo[0])
                elif SampleInfo[1].endswith("R"):
                    R2List.append(filePath)
    df["SampleID"] = IdList
    df["R1"] = R1List
    df["R2"] = R2List
    return df

In [8]:
df = manifest(InDir)

In [9]:
df

,SampleID,R1,R2
0,DA_164,/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__27...,/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__14...
1,DA_24,/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_24__27F...,/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_24__149...
2,DA_251,/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_251__27...,/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_251__14...
3,DA_252,/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_252__27...,/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_252__14...


In [59]:
import collections

In [74]:
SampleList = collections.defaultdict(list)

In [75]:
SampleList

defaultdict(list, {})

In [76]:
for subdir, dirs, files in os.walk(InDir):
    for file in files:
        filePath = os.path.join(subdir, file)
        if file.endswith(".ab1") and os.path.getsize(filePath) > 0:
            info = file.split("__")
            if info[1].endswith("F"):
                SampleList[info[0]].append(filePath)
            elif info[1].endswith("R"):
                SampleList[info[0]].append(filePath)

In [77]:
SampleList

defaultdict(list,
            {'DA_164': ['/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__1492R__F06_2009041343G.ab1',
              '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__27F__F03_2009041343G.ab1'],
             'DA_24': ['/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_24__1492R__E06_2009041339G.ab1',
              '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_24__27F__E03_2009041339G.ab1'],
             'DA_251': ['/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_251__1492R__G06_2009041345G.ab1',
              '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_251__27F__G03_2009041345G.ab1'],
             'DA_252': ['/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_252__1492R__H06_2009041346G.ab1',
              '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_252__27F__H03_2009041346G.ab1']})

## ABI to fastq

In [11]:
ab1 = "/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__1492R__F06_2009041343G.ab1"

In [9]:
ab1 = "/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__27F__F03_2009041343G.ab1"

In [12]:
from Bio import SeqIO
record = SeqIO.parse(ab1, "abi")
count = SeqIO.write(record, "/mnt/d/Lab/16S-Taxa-Phlyo/test_out/DA_164_R2.fastq", "fastq")

In [17]:
from Bio import SeqIO
SeqIO.convert(ab1, "abi", "/mnt/d/Lab/16S-Taxa-Phlyo/test_out/DA_164_R2_convert.fastq", "fastq")

1

In [17]:
def ABI2Fastq(abi, fastq):
    SeqIO.convert(abi, "abi", fastq, "fastq")

In [18]:
def ABI2FastqParallel(abiList, fastqList):
    pool = Pool(processes = 4)
    pool.starmap(ABI2Fastq, zip(abiList, fastqList))
    pool.close()
    pool.join()
    pool.terminate()

In [19]:
OutDir = "/mnt/d/Lab/16S-Taxa-Phlyo/test_out/fastq"

In [20]:
list(df["R1"])

['/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__27F__F03_2009041343G.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_24__27F__E03_2009041339G.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_251__27F__G03_2009041345G.ab1',
 '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_252__27F__H03_2009041346G.ab1']

In [21]:
list(df["R1"].str.replace(".ab1", ".fastq"))

<ipython-input-21-bc2e443c12df>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  list(df["R1"].str.replace(".ab1", ".fastq"))


['/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__27F__F03_2009041343G.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_24__27F__E03_2009041339G.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_251__27F__G03_2009041345G.fastq',
 '/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_252__27F__H03_2009041346G.fastq']

In [22]:
ABI2FastqParallel(df["R1"], df["R1"].str.replace(".ab1", ".fastq"))

<ipython-input-22-90c72fb9313b>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  ABI2FastqParallel(df["R1"], df["R1"].str.replace(".ab1", ".fastq"))


In [23]:
ABI2FastqParallel(df["R2"], df["R2"].str.replace(".ab1", ".fastq"))

<ipython-input-23-5851f60940b7>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  ABI2FastqParallel(df["R2"], df["R2"].str.replace(".ab1", ".fastq"))


### fastqc

In [ ]:
!fastqc DA_164.fastq

## Trimming

### fastp

https://github.com/OpenGene/fastp

In [15]:
!pwd

/mnt/d/Lab/16S-Taxa-Phlyo/pre_data


In [16]:
!cd ../test_out

In [ ]:
!fastp -i DA_164.fastq -o DA_164_p.fastq

In [ ]:
!fastqc DA_164.fastq

In [ ]:
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o trim/DA_164_trim_R1.fastq -O trim/DA_164_trim_R2.fastq

#### trim

In [ ]:
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o trim/DA_164_trim_R1.fastq -O trim/DA_164_trim_R2.fastq --cut_front --cut_tail --cut_window_size 10 --cut_mean_quality 30

In [ ]:
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o trim/DA_164_trim_R1.fastq -O trim/DA_164_trim_R2.fastq --cut_front --cut_tail --cut_front_window_size 10 --cut_front_mean_quality 36

In [ ]:
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o trim/DA_164_trim_R1.fastq -O trim/DA_164_trim_R2.fastq --trim_front1 30 --trim_tail1 100 --max_len1 750 --trim_front2 30 --trim_tail2 100 --max_len2 750

```shell
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o trim/DA_164_trim_R1.fastq -O trim/DA_164_trim_R2.fastq \
--trim_front1 30 --max_len1 750 --trim_front2 30 --max_len2 750 \
--cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30 \
--merge --merged_out trim/DA_164_merge.fastq\
--correction --overlap_len_require 30 --html DA_164.html --report_title DA_164-fastq-merge
```

In [24]:
def RunFastp(R1, R2, prefix, OutDir):
    trimDir = os.path.join(OutDir, "trim")
    mergeDir = os.path.join(OutDir, "merge")
    fastpDir = os.path.join(OutDir, "fastp")
    if os.path.exists(OutDir) == 0:
        os.makedirs(OutDir, 0o777, True)
    if os.path.exists(trimDir) == 0:
        os.makedirs(trimDir, 0o777, True)
    if os.path.exists(mergeDir) == 0:
        os.makedirs(mergeDir, 0o777, True)
    if os.path.exists(fastpDir) == 0:
        os.makedirs(fastpDir, 0o777, True)
    cmd = "fastp -i " + R1 + " -I " + R2 + " -o " + os.path.join(trimDir, prefix + "_trim_R1.fastq") + " -O " + os.path.join(trimDir, prefix + "_trim_R2.fastq") + \
    " --trim_front1 30 --max_len1 750 --trim_front2 30 --max_len2 750 --cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30" + \
    " --merge --merged_out " + os.path.join(mergeDir, prefix + ".fastq") + " --correction --overlap_len_require 20 --html " + os.path.join(fastpDir, prefix + ".html") + \
    " --report_title " + prefix + "-fastq-merge-report"
    #print(cmd)
    subprocess.call(cmd, shell=True)

In [25]:
def RunFastpParallel(R1List, R2List, prefixList, OutDir):
    pool = Pool(processes = 4)
    pool.starmap(RunFastp, zip(R1List, R2List, prefixList, repeat(OutDir)))
    pool.close()
    pool.join()
    pool.terminate()

In [26]:
R1List = df["R1"].str.replace(".ab1", ".fastq")

<ipython-input-26-c3ce5bf156a2>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  R1List = df["R1"].str.replace(".ab1", ".fastq")


In [27]:
R2List = df["R2"].str.replace(".ab1", ".fastq")

<ipython-input-27-38ecb3b25c17>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  R2List = df["R2"].str.replace(".ab1", ".fastq")


In [28]:
prefixList = df["SampleID"]

In [29]:
OutDir = "/mnt/d/Lab/16S-Taxa-Phlyo/test_out"

In [30]:
RunFastpParallel(R1List, R2List, prefixList, OutDir)

In [117]:
R1 = df["R1"].str.replace(".ab1", ".fastq")[0]

<ipython-input-117-38d7a1cd8c22>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  R1 = df["R1"].str.replace(".ab1", ".fastq")[0]


In [119]:
R2 = df["R2"].str.replace(".ab1", ".fastq")[0]

<ipython-input-119-c62cf4f7ba8b>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  R2 = df["R2"].str.replace(".ab1", ".fastq")[0]


In [118]:
R1

'/mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__27F__F03_2009041343G.fastq'

In [120]:
prefix = "DA_164"

In [128]:
RunFastp(R1, R2, prefix, OutDir)

fastp -i /mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__27F__F03_2009041343G.fastq -I /mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__1492R__F06_2009041343G.fastq -o /mnt/d/Lab/16S-Taxa-Phlyo/test_out/trim/DA_164_R1.fastq -O /mnt/d/Lab/16S-Taxa-Phlyo/test_out/trim/DA_164_R2.fastq --trim_front1 30 --max_len1 750 --trim_front2 30 --max_len2 750 --cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30 --merge --merged_out /mnt/d/Lab/16S-Taxa-Phlyo/test_out/merge/DA_164.fastq --correction --overlap_len_require 30 --html /mnt/d/Lab/16S-Taxa-Phlyo/test_out/fastp/DA_164.html --report_title DA_164-fastq-merge-report


```shell
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o trim/DA_164_trim_R1.fastq -O trim/DA_164_trim_R2.fastq \
--trim_front1 30 --max_len1 750 --trim_front2 30 --max_len2 750 \
--cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30 \
--merge --merged_out trim/DA_164_merge.fastq\
--correction --overlap_len_require 30 --html DA_164.html --report_title DA_164-fastq-merge
```

```shell
fastp -i /mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__27F__F03_2009041343G.fastq -I /mnt/d/Lab/16S-Taxa-Phlyo/test_data/DA_164__1492R__F06_2009041343G.fastq -o /mnt/d/Lab/16S-Taxa-Phlyo/test_out/trim/DA_164_R1.fastq -O /mnt/d/Lab/16S-Taxa-Phlyo/test_out/trim/DA_164_R2.fastq --trim_front1 30 --max_len1 750 --trim_front2 30 --max_len2 750 --cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30 --merge --merged_out /mnt/d/Lab/16S-Taxa-Phlyo/test_out/merge/DA_164.fastq --correction --overlap_len_require 30 --html /mnt/d/Lab/16S-Taxa-Phlyo/test_out/fastp/DA_164.html --report_title DA_164-fastq-merge-report
```

路径没问题，那就是fastq转换有问题了？

```shell
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o /mnt/d/Lab/16S-Taxa-Phlyo/test_out/trim/DA_164_R1.fastq -O /mnt/d/Lab/16S-Taxa-Phlyo/test_out/trim/DA_164_R2.fastq --trim_front1 30 --max_len1 750 --trim_front2 30 --max_len2 750 --cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30 --merge --merged_out /mnt/d/Lab/16S-Taxa-Phlyo/test_out/merge/DA_164.fastq --correction --overlap_len_require 30 --html /mnt/d/Lab/16S-Taxa-Phlyo/test_out/fastp/DA_164.html --report_title DA_164-fastq-merge-report
```

In [ ]:
--unpaired1
--unpaired2

In [ ]:
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o trim/DA_164_trim_R1.fastq -O trim/DA_164_trim_R2.fastq --cut_front --cut_tail --cut_window_size 20 --cut_mean_quality 30

In [ ]:
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o trim/DA_164_trim_R1.fastq -O trim/DA_164_trim_R2.fastq --qualified_quality_phred 30

In [ ]:
#ERROR: the mean quality requirement for cutting by quality (--cut_mean_quality) should be 1 ~ 30, suggest 15 ~ 20.

前面的质量还不是很好

In [ ]:
fastp -i DA_164_R1.fastq -I DA_164_R2.fastq -o trim/DA_164_trim_R1.fastq -O trim/DA_164_trim_R2.fastq

In [ ]:
output:--html --report_title string

#### base correction for PE data

In [ ]:
--correction

#### merge paired-end reads

In [ ]:
If you want to trim the reads to maximum length, you can specify -b, --max_len1 for read1, and -B, --max_len2 for read2.

```shell
-O, --out2                           read2 output file name (string [=])
      --unpaired1                      for PE input, if read1 passed QC but read2 not, it will be written to unpaired1. Default is to discard it. (string [=])
      --unpaired2                      for PE input, if read2 passed QC but read1 not, it will be written to unpaired2. If --unpaired2 is same as --unpaired1 (default mode), both unpaired reads will be written to this same file. (string [=])
      --failed_out                     specify the file to store reads that cannot pass the filters. (string [=])
      --overlapped_out                 for each read pair, output the overlapped region if it has no any mismatched base. (string [=])
-m, --merge                          for paired-end input, merge each pair of reads into a single read if they are overlapped. The merged reads will be written to the file given by --merged_out, the unmerged reads will be written to the files specified by --out1 and --out2. The merging mode is disabled by default.
      --merged_out                     in the merging mode, specify the file name to store merged output, or specify --stdout to stream the merged output (string [=])
      --include_unmerged               in the merging mode, write the unmerged or unpaired reads to the file specified by --merge. Disabled by default.
```